In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import lightgbm as lgb
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

sns.set_style('darkgrid')
%matplotlib inline

# Any results you write to the current directory are saved as output.

In [ ]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

In [ ]:
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
train_df.nunique()

In [ ]:
%%time
train_df["date"] = pd.to_datetime(train_df.date)
train_df["year"] = train_df['date'].dt.year
train_df["month"] = train_df['date'].dt.month
train_df["day"] = train_df['date'].dt.day
train_df["dayofweek"] = train_df["date"].dt.dayofweek
#0 : Monday, 6: Sunday in dayofweek

In [ ]:
%%time
test_df["date"] = pd.to_datetime(test_df.date)
test_df["year"] = test_df['date'].dt.year
test_df["month"] = test_df['date'].dt.month
test_df["day"] = test_df['date'].dt.day
test_df["dayofweek"] = test_df["date"].dt.dayofweek
#0 : Monday, 6: Sunday in dayofweek

store : 매장 id  
item : 상품 id  
sales : 해당 날짜에 상품이 팔린 갯수  
date : !!holyday effect는 없음

In [ ]:
tmp_df = train_df.groupby(["year","month","store","item"]).sum().reset_index()[["year","month","store","item","sales"]]

plt.figure(figsize=(20,12))
sns.pointplot(data=tmp_df[(tmp_df.year == 2013) & (tmp_df.store == 1)], x="item", y="sales", hue="month")

1-store의 2013년 월별 item판매량 추이 확인. 거의 모든 item이 비슷한 형태를 보임

In [ ]:
item_list = train_df.item.unique().tolist()
split_item_list = [item_list[x:x+10] for x in range(0,len(item_list),10)]
split_item_list

In [ ]:
tmp_df = train_df[train_df.item.isin(split_item_list[2])].groupby(["year","month","store","item"]).sum().reset_index()[["year","month","store","item","sales"]]
plt.figure(figsize=(16,6))
sns.pointplot(data=tmp_df[(tmp_df.year == 2013) & (tmp_df.store == 1)], x="item", y="sales", hue="month")

In [ ]:
train_df["season"] = train_df.month.map({1:"Winter", 2:"Winter", 3:"Spring", 4:"Spring",5:"Spring",6:"Summer",
                                          7:"Summer", 8:"Summer", 9:"Fall", 10:"Fall", 11:"Fall", 12:"Winter"})
test_df["season"] = test_df.month.map({1:"Winter", 2:"Winter", 3:"Spring", 4:"Spring",5:"Spring",6:"Summer",
                                          7:"Summer", 8:"Summer", 9:"Fall", 10:"Fall", 11:"Fall", 12:"Winter"})

계절변수 생성. 4계절 기준

계절별 판매량 확인

In [ ]:
tmp_df = train_df[train_df.item.isin(split_item_list[0])].groupby(["year","season","store","item"]).sum().reset_index()[["year","season","store","item","sales"]]
plt.figure(figsize=(12,6))
sns.pointplot(data=tmp_df[(tmp_df.year == 2013) & (tmp_df.store == 1)], x="item", y="sales", hue="season")

In [ ]:
tmp_df = train_df[train_df.item.isin(split_item_list[1])].groupby(["year","season","store","item"]).sum().reset_index()[["year","season","store","item","sales"]]
plt.figure(figsize=(12,6))
sns.pointplot(data=tmp_df[(tmp_df.year == 2013) & (tmp_df.store == 1)], x="item", y="sales", hue="season")

In [ ]:
tmp_df = train_df[train_df.item.isin(split_item_list[2])].groupby(["year","season","store","item"]).sum().reset_index()[["year","season","store","item","sales"]]
plt.figure(figsize=(12,6))
sns.pointplot(data=tmp_df[(tmp_df.year == 2013) & (tmp_df.store == 1)], x="item", y="sales", hue="season")

In [ ]:
tmp_df = train_df[train_df.item.isin(split_item_list[3])].groupby(["year","season","store","item"]).sum().reset_index()[["year","season","store","item","sales"]]
plt.figure(figsize=(12,6))
sns.pointplot(data=tmp_df[(tmp_df.year == 2013) & (tmp_df.store == 1)], x="item", y="sales", hue="season")

In [ ]:
tmp_df = train_df[train_df.item.isin(split_item_list[4])].groupby(["year","season","store","item"]).sum().reset_index()[["year","season","store","item","sales"]]
plt.figure(figsize=(12,6))
sns.pointplot(data=tmp_df[(tmp_df.year == 2013) & (tmp_df.store == 1)], x="item", y="sales", hue="season")

봄, 가을엔 거의 같은 판매량을 보임. 여름에 판매량이 많고 겨울에 판매량이 적음

In [ ]:
sns.boxplot(data =train_df, x="dayofweek", y='sales')

주말에 더 많이 팔리는 경향  
큰 차이는 없어 보이지만 그래도 요일별, 적어도 주말과 주중간에는 어느정도 차이가 있어보임  
주말과 주중으로 나누지 않고 요일 그대로 유지한채로 모델에 넣을듯  
or 주말,주중 나누는 column을 추가하여 둘 다 넣어보고 성능과 variance importance차이를 볼 예정 

In [ ]:
"""train_df["is_weekend"] = train_df.dayofweek.map({0:0, 1:0, 2:0, 3:0, 4:0, 5:1, 6:1})
test_df["is_weekend"] = test_df.dayofweek.map({0:0, 1:0, 2:0, 3:0, 4:0, 5:1, 6:1})"""

In [ ]:
train_df.head()

In [ ]:
sales_by_year = train_df[["year","item","sales"]].groupby(["year","item"]).sum().reset_index().pivot("item","year","sales").reset_index()
sales_by_year

In [ ]:
abs((np.array(sales_by_year[2013])-np.array(sales_by_year[2017]))/np.array(sales_by_year[2013]))/4 * 100

연평균 판매량 증가율

In [ ]:
increase_13_to_14 = abs((np.array(sales_by_year[2013])-np.array(sales_by_year[2014]))/np.array(sales_by_year[2013])).mean()
increase_14_to_15 = abs((np.array(sales_by_year[2014])-np.array(sales_by_year[2015]))/np.array(sales_by_year[2014])).mean()
increase_15_to_16 = abs((np.array(sales_by_year[2015])-np.array(sales_by_year[2016]))/np.array(sales_by_year[2015])).mean()
increase_16_to_17 = abs((np.array(sales_by_year[2016])-np.array(sales_by_year[2017]))/np.array(sales_by_year[2016])).mean()

연도별 판매량 증가율

In [ ]:
print("13_to_14 : {}, 14_to_15 : {}\n15_to_16 : {}, 16_to_17 : {}".format(increase_13_to_14, increase_14_to_15, increase_15_to_16, increase_16_to_17))

In [ ]:
train_df['increase'] = train_df.year.map({2013:1, 2014:1+increase_13_to_14, 2015:1+increase_14_to_15, 2016:1+increase_15_to_16, 2017:1+increase_16_to_17})
test_df['increase'] = 0.05

Test_df의 increase는 임의로 정함

In [ ]:
sales_by_StoreAndItem = train_df[["store","item","sales"]].groupby(["store","item"]).sum().reset_index().sort_values(by="sales", ascending=False)

store, item별 총판매량 순위

In [ ]:
sales_by_StoreAndItem["store_item_index"] = pd.cut(train_df[["store","item","sales"]].groupby(["store","item"]).sum().reset_index().sort_values(by="sales", ascending=False).sales,10, labels=range(1,11))

총판매량을 10개의 구간으로 나누어 1~10의 index 부여

In [ ]:
train_df = pd.DataFrame.merge(train_df, sales_by_StoreAndItem.drop("sales", axis=1).reset_index().drop("index", axis=1) , on=["store","item"])
test_df = pd.DataFrame.merge(test_df,sales_by_StoreAndItem.drop("sales", axis=1).reset_index().drop("index", axis=1) , on=["store","item"])

In [ ]:
train_df[["store_item_index"]].groupby("store_item_index").size()

In [ ]:
train_df = train_df.set_index("date")
test_df = test_df.set_index("date")

In [ ]:
train_df["season"] = train_df.season.map({"Winter":4, "Spring":1, "Summer":2, "Fall":3})
test_df["season"] = test_df.season.map({"Winter":4, "Spring":1, "Summer":2, "Fall":3})

lightgbm모델의 input변수로 numeric type을 넣어줘야하기 때문에 계절변수를 numeric type으로 변환

In [ ]:
def smape(actual, target):
    return 100 * np.mean(2 * np.abs(actual - target)/(np.abs(actual) + np.abs(target)))

In [ ]:
model = lgb.LGBMRegressor(n_jobs=-1, n_estimators=2000, max_depth=8, objective='regression_l1', random_state=123)

In [ ]:
valid_df = train_df[(train_df.year == 2017) & (train_df.month.isin([10, 11, 12]))]

In [ ]:
%%time
train_df_dropped = train_df.drop(valid_df.index, axis=0)

In [ ]:
X = train_df_dropped.drop("sales", axis=1)
y= train_df_dropped['sales']
valid_X = valid_df.drop("sales", axis=1)
valid_y = valid_df['sales']

In [ ]:
model.fit(X, y, eval_set=[(valid_X, valid_y)], eval_metric=['mape', smape])

In [ ]:
feature_importances = pd.DataFrame({'importance': model.feature_importances_, 'name': X.columns})
sns.barplot( data=feature_importances.sort_values('importance', ascending=False), x='importance', y='name')

In [ ]:
pred = model.predict(valid_X)
smape(valid_y, pred)

In [ ]:
test_df = test_df.drop("id", axis=1)
pred = model.predict(test_df)

In [ ]:
f = pd.DataFrame({'sales' : pred }, index=range(0,len(pred))).reset_index()
f.columns = ["id", "sales"]
f.to_csv("submission_demandForecasting.csv", index=False)